<a href="https://colab.research.google.com/github/stevenAnto/llm/blob/main/LangChaing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Ingresa tu clave de OpenAI: ")


KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install langchain langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
llm.invoke("la profesion de IA tiene un futuro prometedor?")

In [ ]:
from  langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Eres un asistente de IA, puedes responder a preguntas sobre IA"),
    ("user","{input}")
])

In [ ]:
chain  = prompt | llm
chain.invoke({"input":"la profesion de IA tiene un futuro prometedor?"})

#Memory

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI


In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un asistente de {ability}, puedes responder a preguntas sobre {ability}",
     ),
    MessagesPlaceholder(variable_name="history"),#ingresa el historico de la conversacion
    ("human","{input}")
])

In [ ]:
runnable = prompt | model

In [ ]:
!pip install langchain-community


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [ ]:
store = {}

In [ ]:
#Para saber si hay contexto anterior
def get_session_history(session_id: str)->BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

In [ ]:
with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_message_key="input",
    history_messages_key="history",
)

In [ ]:
with_message_history.invoke(
    {"ability":"ciencia de datos","input":"que es una funcion de perdida"},
    config={"configurable":{"session_id":"abc123"}},
    )

In [ ]:
with_message_history.invoke(
    {"ability":"ciencia de datos","input":"como?, no te entendi"},
    config={"configurable":{"session_id":"abc123"}},
    )

#Tools


In [ ]:
!pip install wikipedia


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
tool.name

In [ ]:
tool.description

In [ ]:
tool.args

In [ ]:
tool.run({"query":"Alan Turing"})

In [ ]:
tool.run({"query":"quien es Alan Turing"})

#Agents


In [ ]:
!pip install -U langchain-tavily


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
api_key = getpass(" Ingresa tu Tavily API Key: ")
search = TavilySearchResults(tavily_api_key=api_key)

In [ ]:
search.invoke("Que temperatura hay en Londres")

In [ ]:
tools = [search, tool]

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

In [ ]:
from langchain import hub

In [ ]:
#prompt de la comunidad
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
from langchain.agents import create_tool_calling_agent

In [ ]:
#Creacion de agentes
agent = create_tool_calling_agent(llm,tools,prompt=prompt)

In [ ]:
from langchain.agents import AgentExecutor

In [ ]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [ ]:
agent_executor.invoke({"input":"hola"})

In [ ]:
agent_executor.invoke({"input":"cual es el mejor  superheroe de Marvel y por que?"})

In [ ]:
agent_executor.invoke({"input":"Que temperatura hace en Nueva York?"})

#Ejemplos de Interacciones

In [ ]:
agent_executor.invoke({"input":"Busca quien fue Alan Turing"})

In [ ]:
agent_executor.invoke({"input":"Dime un chiste."})

In [ ]:
agent_executor.invoke({"input":"¿Qué está pasando en el mundo de la inteligencia artificial?"})

In [ ]:
agent_executor.invoke({"input":"Que esta pasando entre EEUU e Iran?"})

In [ ]:
agent_executor.invoke({"input":"Quien fue Alan Turing y que fue lo mas grande que hizo, busca en diversas fuentes como paginas webs,etc"})

In [ ]:
agent_executor.invoke({"input":"Cuentame otro chiste"})

In [ ]:
agent_executor.invoke({"input":"Pero no tienes otros chiste"})

In [ ]:
agent_executor.invoke({"input":"si"})

In [ ]:
agent_executor.invoke({"input":"respondeme tres preguntas, busca en internet 1. ¿Qué otros modelos de colores existen para las imagenes?;¿Qué otros cambios se pueden realizar a las imágenes con OpenCV?; 3. ¿Cuál seria el uso del binary threshold en el procesmiento de imagenes?"})

In [ ]:
agent_executor.invoke({"input":'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio '})

In [ ]:
agent_executor.invoke({"input":'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio '})

In [ ]:
agent_executor.invoke({"input":'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio '})

In [ ]:
agent_executor.invoke({"input":'Cual fue la ultima preegunta que te hice '})

In [ ]:
agent_executor.invoke({"input":'puedes recordar  nuestro chat?'})

In [ ]:
agent_executor.invoke({"input":'que  preguntas te he hecho anteriormente'})

In [ ]:
agent_executor.invoke({"input":'te daré 3 numeros: 5,6,7. Por favor suma esos tres numeros'})

In [ ]:
agent_executor.invoke({"input":'te daré 3 numeros: 5,6,7. Por favor suma esos tres numeros'})

In [ ]:
agent_executor.invoke({"input":'Tienes acceso al directorio de la computadora donde se esta ejecutando este programa?'})

In [62]:
!pip uninstall -y langchain-googledrive




Found existing installation: langchain-googledrive 0.3.35
Uninstalling langchain-googledrive-0.3.35:
  Successfully uninstalled langchain-googledrive-0.3.35


In [63]:
%pip install --upgrade --quiet  unstructured langchain-googledrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.9 MB/s eta 0:00:00


In [86]:
%pip install --upgrade --quiet  google-api-python-client google-auth-httplib2 google-auth-oauthlib langchain-community




In [64]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [87]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [88]:
import os
from langchain_googledrive.tools.google_drive.tool import GoogleDriveSearchTool
from langchain_googledrive.utilities.google_drive import GoogleDriveAPIWrapper

In [100]:
os.environ["GOOGLE_ACCOUNT_FILE"] = "/content/miproyectodrive-464014-c08c3b8e7b82.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_FILE
SERVICE_ACCOUNT_FILE = "/content/miproyectodrive-464014-c08c3b8e7b82.json"


In [101]:
folder_id ="1lYk54ck5sSJn5hAGY3XvRwIrXT3msUZr"
#folder_id="root"

In [102]:
import logging

logging.basicConfig(level=logging.INFO)

In [103]:

# By default, search only in the filename.
tool = GoogleDriveSearchTool(
    api_wrapper=GoogleDriveAPIWrapper(
        api_file_path=os.environ["GOOGLE_ACCOUNT_FILE"],
        folder_id=folder_id,
        num_results=2,
        template="gdrive-query-in-folder",  # Search in the body of documents
    )
)

In [106]:
tool.run("pruebaDocumento")

'No document found'

In [93]:
print("=== PRUEBA 1: Configuración básica sin template ===")
try:
    tool1 = GoogleDriveAPIWrapper(
        folder_id=folder_id,
        num_results=10,
        # Sin template específico
    )
    search_tool1 = GoogleDriveSearchTool(api_wrapper=tool1)
    resultados1 = search_tool1.run("pruebaDocumento")
    print("Resultado 1:", resultados1)
except Exception as e:
    print(f"Error 1: {e}")

=== PRUEBA 1: Configuración básica sin template ===
Resultado 1: No document found


In [73]:
print("\n=== PRUEBA 2: Sin folder_id específico ===")
try:
    tool2 = GoogleDriveAPIWrapper(
        num_results=10,
        # Sin folder_id para buscar en todo el drive
    )
    search_tool2 = GoogleDriveSearchTool(api_wrapper=tool2)
    resultados2 = search_tool2.run("pruebaDocumento")
    print("Resultado 2:", resultados2)
except Exception as e:
    print(f"Error 2: {e}")


=== PRUEBA 2: Sin folder_id específico ===
Resultado 2: No document found


In [74]:
print("\n=== PRUEBA 3: Búsqueda más amplia ===")
try:
    tool3 = GoogleDriveAPIWrapper(
        folder_id=folder_id,
        num_results=20,
    )
    search_tool3 = GoogleDriveSearchTool(api_wrapper=tool3)
    # Probar búsquedas diferentes
    resultados3a = search_tool3.run("")  # Búsqueda vacía
    resultados3b = search_tool3.run("documento")  # Palabra más general
    resultados3c = search_tool3.run("prueba")  # Parte del nombre

    print("Búsqueda vacía:", resultados3a)
    print("Búsqueda 'documento':", resultados3b)
    print("Búsqueda 'prueba':", resultados3c)
except Exception as e:
    print(f"Error 3: {e}")


=== PRUEBA 3: Búsqueda más amplia ===
Búsqueda vacía: No document found
Búsqueda 'documento': No document found
Búsqueda 'prueba': No document found


In [75]:
print("\n=== DIAGNÓSTICO: Verificar configuración interna ===")
try:
    # Crear wrapper y verificar su configuración interna
    diagnostic_tool = GoogleDriveAPIWrapper(
        folder_id=folder_id,
        num_results=10,
    )

    # Verificar si tiene acceso al servicio
    print("Configuración del wrapper:")
    print(f"- Folder ID: {getattr(diagnostic_tool, 'folder_id', 'No definido')}")
    print(f"- Num results: {getattr(diagnostic_tool, 'num_results', 'No definido')}")

    # Intentar acceso directo al servicio interno
    if hasattr(diagnostic_tool, '_build_service'):
        service = diagnostic_tool._build_service()
        print("✅ Servicio construido correctamente")
    else:
        print("❌ No se puede construir el servicio")

except Exception as e:
    print(f"Error en diagnóstico: {e}")


=== DIAGNÓSTICO: Verificar configuración interna ===
Configuración del wrapper:
- Folder ID: 1lYk54ck5sSJn5hAGY3XvRwIrXT3msUZr
- Num results: 10
❌ No se puede construir el servicio


In [76]:
print("\n=== VERIFICAR PERMISOS DIRECTOS ===")
# Tu código que funciona
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=["https://www.googleapis.com/auth/drive.metadata.readonly"]
)
drive_service = build("drive", "v3", credentials=creds)

# Verificar que los archivos existen
query = f"'{folder_id}' in parents and trashed = false"
results = drive_service.files().list(
    q=query,
    fields="files(id, name, mimeType, modifiedTime)",
    pageSize=20
).execute()

files = results.get("files", [])
print(f"✅ API directa encuentra {len(files)} archivos")
for file in files:
    print(f"  - {file['name']} (ID: {file['id']})")


=== VERIFICAR PERMISOS DIRECTOS ===
✅ API directa encuentra 2 archivos
  - segundoDocumentodePrueba (ID: 1fcISyBqzHU6ntuFf9hm2ezD9kpVkL0NN_HZiYB1BzpM)
  - pruebaDocumento (ID: 1d6gxozQEM5zfH-UcW1IjZGzo2KZpMOJKbzbTOTofPws)


In [77]:
print("\n=== SOLUCIÓN ALTERNATIVA: Configuración manual ===")
try:
    # Configurar scopes más amplios
    creds_full = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=[
            "https://www.googleapis.com/auth/drive.readonly",
            "https://www.googleapis.com/auth/drive.metadata.readonly",
            "https://www.googleapis.com/auth/documents.readonly"
        ]
    )

    # Crear wrapper con credenciales específicas
    tool_manual = GoogleDriveAPIWrapper(
        folder_id=folder_id,
        num_results=10,
        credentials=creds_full  # Si la clase lo soporta
    )

    search_tool_manual = GoogleDriveSearchTool(api_wrapper=tool_manual)
    resultado_manual = search_tool_manual.run("prueba")
    print("Resultado manual:", resultado_manual)

except Exception as e:
    print(f"Error manual: {e}")


=== SOLUCIÓN ALTERNATIVA: Configuración manual ===
Resultado manual: No document found


In [78]:
class DebugGoogleDriveWrapper(GoogleDriveAPIWrapper):
    """Wrapper con debugging para ver qué está pasando internamente"""

    def search(self, query: str):
        print(f"🔍 Búsqueda iniciada con query: '{query}'")
        print(f"📁 Folder ID configurado: {getattr(self, 'folder_id', 'No definido')}")

        try:
            result = super().search(query)
            print(f"✅ Resultado obtenido: {result}")
            return result
        except Exception as e:
            print(f"❌ Error en búsqueda: {e}")
            import traceback
            traceback.print_exc()
            raise

# Probar con wrapper de debugging
try:
    debug_tool = DebugGoogleDriveWrapper(
        folder_id=folder_id,
        num_results=10,
    )
    debug_search = GoogleDriveSearchTool(api_wrapper=debug_tool)
    resultado_debug = debug_search.run("prueba")
    print("Resultado debug:", resultado_debug)
except Exception as e:
    print(f"Error en debug: {e}")

# ===== POSIBLES SOLUCIONES SEGÚN EL PROBLEMA ===
print("\n=== RESUMEN DE POSIBLES SOLUCIONES ===")
print("""
1. ✅ Variable de entorno correcta: GOOGLE_APPLICATION_CREDENTIALS
2. 🔧 Probar sin template específico
3. 🔧 Probar sin folder_id (buscar en todo el drive)
4. 🔧 Ampliar scopes de permisos
5. 🔧 Verificar versión de langchain-googledrive
6. 🔧 Usar búsquedas más generales ("documento" en lugar de "pruebaDocumento")
""")

Resultado debug: No document found

=== RESUMEN DE POSIBLES SOLUCIONES ===

1. ✅ Variable de entorno correcta: GOOGLE_APPLICATION_CREDENTIALS
2. 🔧 Probar sin template específico
3. 🔧 Probar sin folder_id (buscar en todo el drive)
4. 🔧 Ampliar scopes de permisos
5. 🔧 Verificar versión de langchain-googledrive
6. 🔧 Usar búsquedas más generales ("documento" en lugar de "pruebaDocumento")



In [79]:
print("\n=== CÓDIGO FINAL RECOMENDADO ===")

def configuracion_recomendada():
    """Configuración que debería funcionar"""

    # 1. Variable de entorno correcta
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_FILE

    # 2. Configuración simple sin templates complejos
    wrapper = GoogleDriveAPIWrapper(
        num_results=20,  # Más resultados
        # Sin folder_id primero, para probar en todo el drive
    )

    tool = GoogleDriveSearchTool(api_wrapper=wrapper)

    # 3. Probar diferentes tipos de búsqueda
    searches = [
        "",  # Vacía para listar todo
        "prueba",  # Palabra clave
        "documento",  # Tipo de archivo
        "type:document",  # Filtro específico
    ]

    for search_term in searches:
        try:
            result = tool.run(search_term)
            print(f"Búsqueda '{search_term}': {result}")
            if result != "No document found":
                return tool  # Si encuentra algo, devolver esta configuración
        except Exception as e:
            print(f"Error con '{search_term}': {e}")

    return None
configuracion_recomendada()


=== CÓDIGO FINAL RECOMENDADO ===
Búsqueda '': No document found
Búsqueda 'prueba': No document found
Búsqueda 'documento': No document found
Búsqueda 'type:document': No document found


In [80]:
search_tool = GoogleDriveSearchTool(api_wrapper=tool)

# Búsqueda vacía para listar todo
logging.basicConfig(level=logging.INFO)
resultados = search_tool.run("pruebaDocumento")
print("📁 Archivos encontrados por LangChain:")
print(resultados)

📁 Archivos encontrados por LangChain:
No document found


In [81]:
search_tool.description

"A wrapper around Google Drive Search. Useful for when you need to find a document in google drive. The input should be formatted as a list of entities separated with a space. As an example, a list of keywords is 'hello word'."

In [94]:
# 1. Autenticación
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=["https://www.googleapis.com/auth/drive.metadata.readonly"]
)

# 2. Construir servicio
drive_service = build("drive", "v3", credentials=creds)

# 3. Realizar búsqueda
# Busca archivos dentro del folder que NO estén en la papelera
query = f"'{folder_id}' in parents and trashed = false"

results = drive_service.files().list(
    q=query,
    fields="files(id, name, mimeType, modifiedTime)",
    pageSize=20
).execute()

files = results.get("files", [])
if not files:
    print("❌ No se encontraron archivos.")
else:
    print("✅ Archivos encontrados:")
    for file in files:
        print(f"- {file['name']} ({file['mimeType']}) — modificado: {file['modifiedTime']}")

✅ Archivos encontrados:
- segundoDocumentodePrueba (application/vnd.google-apps.document) — modificado: 2025-06-26T00:47:12.182Z
- pruebaDocumento (application/vnd.google-apps.document) — modificado: 2025-06-25T20:44:38.224Z
